In [ ]:
M, N = 10, 12
M, N = 3, 3
sigma = 2.0

# Extract blurring matrix
eye = np.eye(M*N, M*N)
bmat = np.zeros((M*N,M*N))

for j in range(M*N):
    bmat[j,:] = gaussian_filter(eye[j,:].reshape((M,N)), sigma, mode='wrap').flatten()

plt.imshow(bmat)
plt.show()

In [ ]:
# Check that the matrix is diagonalized by the DFT
m1 = spy.linalg.dft(M, scale='sqrtn')
m2 = spy.linalg.dft(N, scale='sqrtn')
dftmtx = np.kron(m1, m2)
idftmtx = dftmtx.conj().T
D = dftmtx @ bmat @ idftmtx
print(np.linalg.norm(D - np.diag(np.diag(D))))
plt.imshow(np.real(D))
plt.show()

In [ ]:
# Next, we need to compute $\Pi$ like we did earlier with $\Lambda$.
rand_img = np.random.randn(M,N)
mat_prod_rand_img = np.fft.fft2( ( L_mat @ np.fft.ifft2(rand_img, norm='ortho').flatten() ).reshape((M,N)), norm='ortho')
pi = mat_prod_rand_img/rand_img
pi = np.real(pi) # We know it should be real
pi_inv = 1/pi
print(pi)

In [ ]:
# Next, we need to compute $\Pi$ like we did earlier with $\Lambda$.
rand_img = np.random.randn(M,N)
mat_prod_rand_img = np.fft.fft2( blur_op ( np.fft.ifft2(rand_img, norm='ortho').flatten() ).reshape((M,N)), norm='ortho')
pi = mat_prod_rand_img/rand_img
pi = np.real(pi) # We know it should be real
pi_inv = 1/pi
print(pi)

In [ ]:
# Check that dftmtx and fft2 give same result when acting on vectors
r = np.random.randn(M,N)
r1 = ( dftmtx @ r.flatten() ).reshape((M,N))
r2 = np.fft.fft2(r, norm='ortho')
np.linalg.norm(r1 - r2)

In [ ]:
# Fourthly, we can build the sparse differencing matrix for the prior
N1, M1 = 5, 5

d_mat_vert = sp.eye(M1)
d_mat_vert.setdiag(-1,k=-1)
d_mat_vert = sp.csc_matrix(d_mat_vert)
d_mat_vert[0,-1] = -1
eye_horiz = sp.eye(N1)
d_mat_one = sp.kron(eye_horiz, d_mat_vert)

d_mat_horiz = sp.eye(N1)
d_mat_horiz.setdiag(-1,k=-1)
d_mat_horiz = sp.csc_matrix(d_mat_horiz)
d_mat_horiz[0,-1] = -1
eye_vert = sp.eye(M1)
d_mat_two = sp.kron(d_mat_horiz, eye_vert)

d_mat = sp.vstack([d_mat_one, d_mat_two])

prec_mat = d_mat.T @ d_mat

In [ ]:
# Check that the matrix is diagonalized by the DFT
m1 = spy.linalg.dft(M1, scale='sqrtn')
m2 = spy.linalg.dft(N1, scale='sqrtn')
dftmtx = np.kron(m1, m2)
idftmtx = dftmtx.conj().T
D = dftmtx @ prec_mat @ idftmtx
print(np.linalg.norm(D - np.diag(np.diag(D))))
plt.imshow(np.real(D))
plt.show()